In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import numpy as np
import scprinter as scp
import scanpy as sc
import anndata
sc._settings.settings.n_jobs = -1

In [ ]:
work_dir = '/ewsc/rzhang/GAGE_seq_AD/scATAC'

In [2]:
meta = pd.read_csv('snATAC_meta_TSS_6.tsv', sep='\t')
meta

,cellId,Individual,Batch,Celltype,Cluster,msex,ADdiag3types,nFrags,FRIP,ReadsInPeaks,TSSEnrichment
0,D19-13182#CTTGCTGCATCCCAAA-1,ROSMAP-17378,Library11,Ast,C11,0,earlyAD,99510,0.200988,39985,7.848
1,D19-13182#TCGGGACTCACCACAA-1,ROSMAP-17378,Library11,Ast,C11,0,earlyAD,99494,0.275523,54806,9.830
2,D19-13182#TGCTTTATCATGTTCT-1,ROSMAP-17378,Library11,Exc,C15,0,earlyAD,96805,0.280257,54256,6.224
3,D19-13182#GCGAGTCAGTACCACT-1,ROSMAP-17378,Library11,Ast,C11,0,earlyAD,95725,0.363587,69600,7.733
4,D19-13182#GGAACCCTCTCCCTTT-1,ROSMAP-17378,Library11,Ast,C12,0,earlyAD,94837,0.247250,46883,6.601
...,...,...,...,...,...,...,...,...,...,...,...
171114,D19-13184#TATTGCTAGTGATATG-1,ROSMAP-43882,Library11,Oli,C8,1,lateAD,1012,0.322453,652,10.473
171115,D19-13184#TGAATCGAGAGACTCG-1,ROSMAP-43882,Library11,Inh,C22,1,lateAD,1010,0.229663,463,10.608
171116,D19-13184#TATTGCTGTTCTTATC-1,ROSMAP-43882,Library11,Exc,C15,1,lateAD,1003,0.173154,347,6.695
171117,D19-13184#CCAATGATCACATTCT-1,ROSMAP-43882,Library11,Oli,C6,1,lateAD,1002,0.584830,1172,14.488


In [3]:
lib2whitelist = {}
for xx in meta['cellId']:
    xx = xx.split("#")
    lib = xx[0]
    bc = xx[1]
    if lib not in lib2whitelist:
        lib2whitelist[lib] = []
    lib2whitelist[lib].append(bc)

In [4]:
libs = ['D19-8609', 
'D19-12698', 
'D19-12529', 
'D19-13171', 
'D19-12530', 
'D19-13184', 
'D19-8608', 
'D19-12525', 
'D19-12701', 
'D19-12513', 
'D19-13167', 
'D19-125460', 
'D19-8610', 
'D19-13154', 
'D19-125459', 
'D19-13177', 
'D19-12522', 
'D19-13175', 
'D19-125454', 
'D19-12527']
frags = [f'./fragments/{lib}_fragments.bed.gz' for lib in libs]
whitelists = [lib2whitelist[lib] for lib in libs]

In [5]:
work_dir = '/ewsc/zhangruo/GAGE_seq_AD/scATAC'

In [6]:
import scprinter as scp
import time
start = time.time()

if os.path.exists(f'{work_dir}/scprinter_tss6.h5ad'):
    printer = scp.load_printer(f'{work_dir}/scprinter_tss6.h5ad', scp.genome.hg38)
else:
    printer = scp.pp.import_fragments(
                        path_to_frags=frags,
                        barcodes=whitelists,
                        savename=f'{work_dir}/scprinter_tss6.h5ad',
                        genome=scp.genome.hg38,
                        min_num_fragments=0, min_tsse=0,
                        sorted_by_barcode=False, 
                        low_memory=False,
                        sample_names=libs
                        )
print ("takes", time.time()-start)

You are now using the beta auto_detect_shift function, this overwrites the plus_shift and minus_shift you provided
If you believe the auto_detect_shift is wrong, please set auto_detect_shift=False


Importing fragments:   0%|          | 0/20 [00:00<?, ?it/s]

Detecting the shift in the paired end fragments file
If you think the above message is wrong, please check the input file format
Minimum MSE is 0.0017856514509375, shift detected
Minimum MSE is 0.0017965175130125002, shift detected
detected plus_shift and minus_shift are 4 -5 for ./fragments/D19-8609_fragments.bed.gz
Detecting the shift in the paired end fragments file
If you think the above message is wrong, please check the input file format
Minimum MSE is 9.554078218750006e-05, shift detected
Minimum MSE is 0.00010659780676250002, shift detected
detected plus_shift and minus_shift are 4 -5 for ./fragments/D19-12698_fragments.bed.gz
Detecting the shift in the paired end fragments file
If you think the above message is wrong, please check the input file format
Minimum MSE is 0.0001597581021875001, shift detected
Minimum MSE is 0.00011379893176250004, shift detected
detected plus_shift and minus_shift are 4 -5 for ./fragments/D19-12529_fragments.bed.gz
Detecting the shift in the paired

  0%|          | 0/20 [00:00<?, ?it/s]

start transferring insertions
takes 1807.1912803649902


In [7]:
from itertools import chain
all_whitelist = []
for lib, wl in zip(libs, whitelists):
    for x in wl:
        all_whitelist.append(f'{lib}_{x}')

In [8]:
meta.index= [xx.split("#")[0] + "_" + xx.split("#")[1] for xx in meta['cellId']]
meta['index'] = meta.index
meta = meta.loc[all_whitelist].copy()
meta

,cellId,Individual,Batch,Celltype,Cluster,msex,ADdiag3types,nFrags,FRIP,ReadsInPeaks,TSSEnrichment,index
D19-8609_ATTGTCTTCTGGCTAA-1,D19-8609#ATTGTCTTCTGGCTAA-1,ROSMAP-33422,Library2,Exc,C14,1,nonAD,99891,0.325869,64964,6.236,D19-8609_ATTGTCTTCTGGCTAA-1
D19-8609_TATGTTCTCTTCAGAG-1,D19-8609#TATGTTCTCTTCAGAG-1,ROSMAP-33422,Library2,Exc,C14,1,nonAD,98917,0.306965,60587,6.752,D19-8609_TATGTTCTCTTCAGAG-1
D19-8609_CATTCATTCGGAAAGC-1,D19-8609#CATTCATTCGGAAAGC-1,ROSMAP-33422,Library2,Exc,C14,1,nonAD,95683,0.312696,59730,6.747,D19-8609_CATTCATTCGGAAAGC-1
D19-8609_TCAGGGCAGCACGTAG-1,D19-8609#TCAGGGCAGCACGTAG-1,ROSMAP-33422,Library2,Inh,C20,1,nonAD,94457,0.288590,54443,7.978,D19-8609_TCAGGGCAGCACGTAG-1
D19-8609_GTGTCCTAGCCTTTGA-1,D19-8609#GTGTCCTAGCCTTTGA-1,ROSMAP-33422,Library2,Ast,C10,1,nonAD,93968,0.213309,39983,6.631,D19-8609_GTGTCCTAGCCTTTGA-1
...,...,...,...,...,...,...,...,...,...,...,...,...
D19-12527_CGGACCACAGTCAGCC-1,D19-12527#CGGACCACAGTCAGCC-1,ROSMAP-42915,Library7,Oli,C3,0,lateAD,1080,0.421296,910,11.279,D19-12527_CGGACCACAGTCAGCC-1
D19-12527_ACTTTCAGTCTAACCA-1,D19-12527#ACTTTCAGTCTAACCA-1,ROSMAP-42915,Library7,Oli,C3,0,lateAD,1075,0.141860,305,6.178,D19-12527_ACTTTCAGTCTAACCA-1
D19-12527_GATCATGGTTTCCGGG-1,D19-12527#GATCATGGTTTCCGGG-1,ROSMAP-42915,Library7,Oli,C3,0,lateAD,1062,0.322976,686,9.424,D19-12527_GATCATGGTTTCCGGG-1
D19-12527_ATGTCTTAGGGTTCCC-1,D19-12527#ATGTCTTAGGGTTCCC-1,ROSMAP-42915,Library7,Oli,C3,0,lateAD,1033,0.296709,613,8.086,D19-12527_ATGTCTTAGGGTTCCC-1


In [9]:
cell_grouping, group_names = scp.utils.df2cell_grouping(printer, meta[['index', 'Celltype']])

In [10]:
scp.pp.call_peaks(
    printer=printer,
    frag_file=frags,
    cell_grouping=cell_grouping,
    group_names=list(group_names),
    iterative_peak_merging=True,
    merge_across_groups=True,
    preset='chromvar',
    sample_names=libs,
    overwrite=False,
    fdr_threshold=0.001)

running macs2 with macs2 callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/Mic_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n Mic -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01


INFO  @ Sat, 15 Mar 2025 16:03:18: 
# Command line: callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/Mic_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n Mic -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01
# ARGUMENTS LIST:
# name = Mic
# format = BEDPE
# ChIP-seq file = ['/ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/Mic_filtered_frag.tsv.gz']
# control file = None
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 1.00e-02
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragment length "d".
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 10000 bps
# Broad region calling is off
# Paired-End mode is on
# Searching for subpeak summits is on
# MACS will save fragment

running macs2 with macs2 callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/Vas_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n Vas -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01


INFO  @ Sat, 15 Mar 2025 16:04:11: 
# Command line: callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/Vas_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n Vas -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01
# ARGUMENTS LIST:
# name = Vas
# format = BEDPE
# ChIP-seq file = ['/ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/Vas_filtered_frag.tsv.gz']
# control file = None
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 1.00e-02
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragment length "d".
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 10000 bps
# Broad region calling is off
# Paired-End mode is on
# Searching for subpeak summits is on
# MACS will save fragment

running macs2 with macs2 callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/Opc_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n Opc -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01


...

running macs2 with macs2 callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/Oli_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n Oli -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01


INFO  @ Sat, 15 Mar 2025 16:19:00: 
# Command line: callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/Oli_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n Oli -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01
# ARGUMENTS LIST:
# name = Oli
# format = BEDPE
# ChIP-seq file = ['/ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/Oli_filtered_frag.tsv.gz']
# control file = None
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 1.00e-02
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragment length "d".
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 10000 bps
# Broad region calling is off
# Paired-End mode is on
# Searching for subpeak summits is on
# MACS will save fragment

Reading in peak summit file(s):
NOTE: Assuming all start coordinates are 0-based ..

Padding peak summits by: 400 bp on either side for
Removing peaks overlapping with blacklisted regions and out of bound peaks based on chromosome sizes ..



  0%|          | 0/7 [00:00<?, ?it/s]

Filtering overlapping peaks based on peak summit score ..
round: 1 1205342 peaks unresolved 311571 peaks selected
round: 2 321417 peaks unresolved 65448 peaks selected
round: 3 104062 peaks unresolved 19759 peaks selected
round: 4 34704 peaks unresolved 6548 peaks selected
round: 5 11159 peaks unresolved 2078 peaks selected
round: 6 3664 peaks unresolved 682 peaks selected
round: 7 1064 peaks unresolved 220 peaks selected
round: 8 301 peaks unresolved 69 peaks selected
round: 9 55 peaks unresolved 13 peaks selected
round: 10 16 peaks unresolved 6 peaks selected
round: 11 2 peaks unresolved 1 peaks selected
finish clearing
finish sorting
finished summary


In [11]:
df = printer.uns['peak_calling']['merged']
peaks = pd.DataFrame(df, columns=df.columns)
peaks

/tmp/ipykernel_401221/3158716219.py:1: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  df = printer.uns['peak_calling']['merged']


,chrom,start,end,name,score,final_score,cell,rank
0,chr1,817183,817483,Mic_peak_2,49.0784,49.0784,Mic,71146
1,chr1,818121,818421,Opc_peak_3,5.43633,5.43633,Opc,365799
2,chr1,827386,827686,Oli_peak_4,331.329,331.329,Oli,15933
3,chr1,850353,850653,Exc_peak_4,4.58036,4.58036,Exc,262170
4,chr1,869792,870092,Oli_peak_5,66.1156,66.1156,Oli,55696
...,...,...,...,...,...,...,...,...
406390,chrX,155820136,155820436,Oli_peak_112984,11.1663,11.1663,Oli,176104
406391,chrX,155845361,155845661,Ast_peak_112992,5.6359,5.6359,Ast,241527
406392,chrX,155874756,155875056,Exc_peak_156151,10.639,10.639,Exc,179803
406393,chrX,155881107,155881407,Oli_peak_112985,413.806,413.806,Oli,13398


In [12]:
peaks.to_csv(f'{work_dir}/tss6_atac_peaks.bed', sep='\t', header=False, index=False)

In [13]:
scp.pp.call_peaks(
    printer=printer,
    frag_file=frags,
    cell_grouping=cell_grouping,
    group_names=list(group_names),
    iterative_peak_merging=True,
    merge_across_groups=False,
    preset='chromvar',
    sample_names=libs,
    overwrite=False,
    fdr_threshold=0.001)

continue
continue
continue
continue
continue
continue
continue
Reading in peak summit file(s):
NOTE: Assuming all start coordinates are 0-based ..

Padding peak summits by: 400 bp on either side for
Removing peaks overlapping with blacklisted regions and out of bound peaks based on chromosome sizes ..

Filtering overlapping peaks based on peak summit score ..
round: 1 176485 peaks unresolved 104631 peaks selected
round: 2 21244 peaks unresolved 10422 peaks selected
round: 3 3508 peaks unresolved 1603 peaks selected
round: 4 660 peaks unresolved 275 peaks selected
round: 5 159 peaks unresolved 62 peaks selected
round: 6 36 peaks unresolved 17 peaks selected
round: 7 8 peaks unresolved 5 peaks selected
round: 8 1 peaks unresolved 1 peaks selected
finish clearing
finish sorting
finished summary
Reading in peak summit file(s):
NOTE: Assuming all start coordinates are 0-based ..

Padding peak summits by: 400 bp on either side for
Removing peaks overlapping with blacklisted regions and out o

In [16]:
meta

,cellId,Individual,Batch,Celltype,Cluster,msex,ADdiag3types,nFrags,FRIP,ReadsInPeaks,TSSEnrichment,index
D19-8609_ATTGTCTTCTGGCTAA-1,D19-8609#ATTGTCTTCTGGCTAA-1,ROSMAP-33422,Library2,Exc,C14,1,nonAD,99891,0.325869,64964,6.236,D19-8609_ATTGTCTTCTGGCTAA-1
D19-8609_TATGTTCTCTTCAGAG-1,D19-8609#TATGTTCTCTTCAGAG-1,ROSMAP-33422,Library2,Exc,C14,1,nonAD,98917,0.306965,60587,6.752,D19-8609_TATGTTCTCTTCAGAG-1
D19-8609_CATTCATTCGGAAAGC-1,D19-8609#CATTCATTCGGAAAGC-1,ROSMAP-33422,Library2,Exc,C14,1,nonAD,95683,0.312696,59730,6.747,D19-8609_CATTCATTCGGAAAGC-1
D19-8609_TCAGGGCAGCACGTAG-1,D19-8609#TCAGGGCAGCACGTAG-1,ROSMAP-33422,Library2,Inh,C20,1,nonAD,94457,0.288590,54443,7.978,D19-8609_TCAGGGCAGCACGTAG-1
D19-8609_GTGTCCTAGCCTTTGA-1,D19-8609#GTGTCCTAGCCTTTGA-1,ROSMAP-33422,Library2,Ast,C10,1,nonAD,93968,0.213309,39983,6.631,D19-8609_GTGTCCTAGCCTTTGA-1
...,...,...,...,...,...,...,...,...,...,...,...,...
D19-12527_CGGACCACAGTCAGCC-1,D19-12527#CGGACCACAGTCAGCC-1,ROSMAP-42915,Library7,Oli,C3,0,lateAD,1080,0.421296,910,11.279,D19-12527_CGGACCACAGTCAGCC-1
D19-12527_ACTTTCAGTCTAACCA-1,D19-12527#ACTTTCAGTCTAACCA-1,ROSMAP-42915,Library7,Oli,C3,0,lateAD,1075,0.141860,305,6.178,D19-12527_ACTTTCAGTCTAACCA-1
D19-12527_GATCATGGTTTCCGGG-1,D19-12527#GATCATGGTTTCCGGG-1,ROSMAP-42915,Library7,Oli,C3,0,lateAD,1062,0.322976,686,9.424,D19-12527_GATCATGGTTTCCGGG-1
D19-12527_ATGTCTTAGGGTTCCC-1,D19-12527#ATGTCTTAGGGTTCCC-1,ROSMAP-42915,Library7,Oli,C3,0,lateAD,1033,0.296709,613,8.086,D19-12527_ATGTCTTAGGGTTCCC-1


In [17]:
cell_grouping, group_names = scp.utils.df2cell_grouping(printer, meta[meta['ADdiag3types'] == 'nonAD'][['index', 'Celltype']])
group_names = [f'CT_{x}' for x in group_names]

In [18]:
scp.pp.call_peaks(
    printer=printer,
    frag_file=frags,
    cell_grouping=cell_grouping,
    group_names=list(group_names),
    iterative_peak_merging=True,
    merge_across_groups=False,
    preset='chromvar',
    sample_names=libs,
    overwrite=False,
    fdr_threshold=0.001)

running macs2 with macs2 callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/CT_Mic_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n CT_Mic -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01


INFO  @ Sat, 15 Mar 2025 17:37:44: 
# Command line: callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/CT_Mic_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n CT_Mic -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01
# ARGUMENTS LIST:
# name = CT_Mic
# format = BEDPE
# ChIP-seq file = ['/ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/CT_Mic_filtered_frag.tsv.gz']
# control file = None
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 1.00e-02
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragment length "d".
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 10000 bps
# Broad region calling is off
# Paired-End mode is on
# Searching for subpeak summits is on
# MACS will s

running macs2 with macs2 callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/CT_Vas_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n CT_Vas -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01


INFO  @ Sat, 15 Mar 2025 17:37:45: 
# Command line: callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/CT_Vas_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n CT_Vas -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01
# ARGUMENTS LIST:
# name = CT_Vas
# format = BEDPE
# ChIP-seq file = ['/ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/CT_Vas_filtered_frag.tsv.gz']
# control file = None
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 1.00e-02
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragment length "d".
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 10000 bps
# Broad region calling is off
# Paired-End mode is on
# Searching for subpeak summits is on
# MACS will s

running macs2 with macs2 callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/CT_Opc_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n CT_Opc -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01


...

running macs2 with

INFO  @ Sat, 15 Mar 2025 17:42:58:  17000000 


 macs2 callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/CT_Oli_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n CT_Oli -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01


INFO  @ Sat, 15 Mar 2025 17:42:58: 
# Command line: callpeak --nomodel -t /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/CT_Oli_filtered_frag.tsv.gz --outdir /ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/macs2 -n CT_Oli -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01
# ARGUMENTS LIST:
# name = CT_Oli
# format = BEDPE
# ChIP-seq file = ['/ewsc/zhangruo/GAGE_seq_AD/scATAC/scprinter_tss6_supp/CT_Oli_filtered_frag.tsv.gz']
# control file = None
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 1.00e-02
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragment length "d".
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 10000 bps
# Broad region calling is off
# Paired-End mode is on
# Searching for subpeak summits is on
# MACS will s

Reading in peak summit file(s):
NOTE: Assuming all start coordinates are 0-based ..

Padding peak summits by: 400 bp on either side for
Removing peaks overlapping with blacklisted regions and out of bound peaks based on chromosome sizes ..

Filtering overlapping peaks based on peak summit score ..
round: 1 149405 peaks unresolved 92347 peaks selected
round: 2 15704 peaks unresolved 7972 peaks selected
round: 3 2382 peaks unresolved 1122 peaks selected
round: 4 403 peaks unresolved 183 peaks selected
round: 5 80 peaks unresolved 35 peaks selected
round: 6 15 peaks unresolved 9 peaks selected
finish clearing
finish sorting
finished summary
Reading in peak summit file(s):
NOTE: Assuming all start coordinates are 0-based ..

Padding peak summits by: 400 bp on either side for
Removing peaks overlapping with blacklisted regions and out of bound peaks based on chromosome sizes ..

Filtering overlapping peaks based on peak summit score ..
round: 1 259907 peaks unresolved 154196 peaks selected


In [22]:
for key in printer.uns['peak_calling'].keys():
    if "cleaned" in key:
        df = printer.uns['peak_calling'][key]
        peaks = pd.DataFrame(df, columns=df.columns)
        # peaks.to_csv(f'{work_dir}/tss6_{key}_atac_peaks.bed', sep='\t', header=False, index=False)
        atac_adata = scp.pp.make_peak_matrix(printer=printer,
                                     regions=peaks,
                                     region_width=300,
                                     sparse=True)
        atac_adata.obs = meta.loc[atac_adata.obs.index].copy()
        atac_adata.write(f'{key}_cellxpeak.h5ad')

/tmp/ipykernel_401221/3531364185.py:1: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  for key in printer.uns['peak_calling'].keys():
/tmp/ipykernel_401221/3531364185.py:3: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  df = printer.uns['peak_calling'][key]


Making peak matrix:   0%|          | 0/10849 [00:00<?, ?it/s]

/tmp/ipykernel_401221/3531364185.py:3: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  df = printer.uns['peak_calling'][key]


Making peak matrix:   0%|          | 0/80325 [00:00<?, ?it/s]

/tmp/ipykernel_401221/3531364185.py:3: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  df = printer.uns['peak_calling'][key]


...

Making peak matrix:   0%|          | 0/172346 [00:00<?, ?it/s]

/tmp/ipykernel_401221/3531364185.py:3: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  df = printer.uns['peak_calling'][key]


Making peak matrix:   0%|          | 0/113945 [00:00<?, ?it/s]

/tmp/ipykernel_401221/3531364185.py:3: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  df = printer.uns['peak_calling'][key]


Making peak matrix:   0%|          | 0/117016 [00:00<?, ?it/s]